# hybrid_model_optimized_final_fake

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input, mixed_precision
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, accuracy_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

# Load the DataFrame
file_path = r"C:\Users\meh91075\Downloads\merged_features_filtered.csv"
df = pd.read_csv(file_path)
print(f"Original CSV data loaded. Number of rows: {len(df)}")
print(f"Original distribution of labels:")
print(df['label_majority'].value_counts())

# Optimize GPU configuration
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        print(e)

# Activate Mixed Precision
if tf.test.is_built_with_cuda():
    policy = mixed_precision.Policy('mixed_float16')
    mixed_precision.set_global_policy(policy)
    print("Mixed precision policy set to mixed_float16")
else:
    print("CUDA not available, using default precision")

# Paths and Configuration
base_dir = "C:/Users/meh91075/Desktop/02_AutoSplit/02_AutoSplit/10_Datasets/01_Autosplit_dataset"
base_dir1 = "C:/Users/meh91075/Downloads/04_Standardparts"
filtered_dir = os.path.join(base_dir1, 'png')
non_filtered_dir = os.path.join(base_dir, 'New_non_filtered_data')

img_height, img_width = 224, 224
batch_size = 32

def get_image_paths(directory):
    return [os.path.join(root, file) for root, _, files in os.walk(directory) 
            for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

filtered_images = get_image_paths(filtered_dir)
non_filtered_images = get_image_paths(non_filtered_dir)
print(f"Found images: Filtered: {len(filtered_images)}, Not filtered: {len(non_filtered_images)}")

def extract_filename(path):
    basename = os.path.basename(path)
    return basename.replace('_ISO', '').replace('_ISOAlt', '').split('.')[0]

image_df = pd.DataFrame({
    'image_path': filtered_images + non_filtered_images,
    'label': [1] * len(filtered_images) + [0] * len(non_filtered_images),
    'filename': [extract_filename(path) for path in filtered_images + non_filtered_images]
})

print("Sample of extracted filenames:")
print(image_df['filename'].head())

# Adjusted merge process
merged_df = pd.merge(df, image_df[['filename', 'image_path']], on='filename', how='left')
merged_df['label'] = merged_df['label_majority'].map({1: 0, 2: 0, 3: 0, 4: 1})
merged_df['has_image'] = merged_df['image_path'].notna()

# Filter only entries with associated images
merged_df_with_images = merged_df[merged_df['has_image']]

# Undersampling of the majority class
majority_class = merged_df_with_images[merged_df_with_images['label'] == 0]
minority_class = merged_df_with_images[merged_df_with_images['label'] == 1]

majority_downsampled = majority_class.sample(n=2000, random_state=42)
balanced_df = pd.concat([majority_downsampled, minority_class])

print(f"After undersampling: Number of rows: {len(balanced_df)}")
print("Distribution of labels after undersampling:")
print(balanced_df['label'].value_counts())

# Train-Test-Split
train_df, val_df = train_test_split(balanced_df, test_size=0.2, stratify=balanced_df['label'], random_state=42)
print(f"Training data: {len(train_df)}, Validation data: {len(val_df)}")

# Determine the numeric columns
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_columns = [col for col in numeric_columns if col not in ['label_majority', 'label']]

# Normalization of numeric data
scaler = StandardScaler()
train_df[numeric_columns] = scaler.fit_transform(train_df[numeric_columns])
val_df[numeric_columns] = scaler.transform(val_df[numeric_columns])

# Optimized data generator function with data augmentation
def create_dataset(dataframe, is_training=True):
    def parse_function(filename, label, *numeric_data):
        img = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [img_height, img_width])
        img = tf.keras.applications.resnet_v2.preprocess_input(img)
        
        if is_training:
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_brightness(img, max_delta=0.2)
            img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
            img = tf.image.random_saturation(img, lower=0.8, upper=1.2)
            img = tf.image.random_hue(img, max_delta=0.1)
        
        numeric_data = tf.convert_to_tensor(numeric_data, dtype=tf.float32)
        numeric_data = tf.squeeze(numeric_data)
        return (img, numeric_data), label

    dataset = tf.data.Dataset.from_tensor_slices((
        dataframe['image_path'].values,
        dataframe['label'].values,
        dataframe[numeric_columns].values
    ))
    
    dataset = dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    
    if is_training:
        dataset = dataset.shuffle(buffer_size=len(dataframe)).repeat()
    else:
        dataset = dataset.repeat()
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

# Create the datasets
train_dataset = create_dataset(train_df)
val_dataset = create_dataset(val_df, is_training=False)

# Calculate steps per epoch
steps_per_epoch = len(train_df) // batch_size
validation_steps = len(val_df) // batch_size

# Optimized model definition
def create_optimized_hybrid_model(img_height, img_width, num_numeric_features):
    # Image processing branch (CNN)
    img_input = Input(shape=(img_height, img_width, 3))
    
    # Use a pre-trained model for feature extraction
    base_model = tf.keras.applications.ResNet50V2(include_top=False, weights='imagenet', input_tensor=img_input)
    
    # Freeze the pre-trained layers
    base_model.trainable = False
    
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    # Numerical data branch (MLP)
    num_input = Input(shape=(num_numeric_features,))
    y = layers.Dense(128, activation='relu')(num_input)
    y = layers.BatchNormalization()(y)
    y = layers.Dropout(0.3)(y)
    y = layers.Dense(256, activation='relu')(y)
    y = layers.BatchNormalization()(y)
    y = layers.Dropout(0.4)(y)
    y = layers.Dense(128, activation='relu')(y)
    y = layers.BatchNormalization()(y)
    y = layers.Dropout(0.3)(y)
    
    # Combining the branches
    combined = layers.concatenate([x, y])
    
    # Final layers
    z = layers.Dense(256, activation='relu')(combined)
    z = layers.BatchNormalization()(z)
    z = layers.Dropout(0.5)(z)
    z = layers.Dense(128, activation='relu')(z)
    z = layers.BatchNormalization()(z)
    z = layers.Dropout(0.5)(z)
    
    # Output layer
    output = layers.Dense(1, activation='sigmoid')(z)
    
    model = models.Model(inputs=[img_input, num_input], outputs=output)
    return model

# Learning Rate Logger Callback
class LearningRateLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            lr = self.model.optimizer.learning_rate
            if isinstance(lr, tf.keras.optimizers.schedules.LearningRateSchedule):
                lr = lr(self.model.optimizer.iterations)
            logs['lr'] = tf.keras.backend.get_value(lr)

# Custom ReduceLROnPlateau Callback
class CustomReduceLROnPlateau(tf.keras.callbacks.Callback):
    def __init__(self, factor=0.1, patience=10, min_lr=0, monitor='val_loss', mode='min'):
        super(CustomReduceLROnPlateau, self).__init__()
        self.factor = factor
        self.patience = patience
        self.min_lr = min_lr
        self.monitor = monitor
        self.mode = mode
        self.best = np.Inf if mode == 'min' else -np.Inf
        self.wait = 0
        self.best_weights = None

    def on_train_begin(self, logs=None):
        self.best = np.Inf if self.mode == 'min' else -np.Inf
        self.wait = 0
        self.best_weights = None

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        current = logs.get(self.monitor)
        if current is None:
            return
        
        if self.mode == 'min' and current < self.best:
            self.best = current
            self.wait = 0
            self.best_weights = self.model.get_weights()
        elif self.mode == 'max' and current > self.best:
            self.best = current
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
        
        if self.wait >= self.patience:
            current_lr = self.model.optimizer.learning_rate
            if isinstance(current_lr, tf.keras.optimizers.schedules.LearningRateSchedule):
                current_lr = current_lr(self.model.optimizer.iterations)
            current_lr = float(current_lr.numpy())
            
            if current_lr > self.min_lr:
                new_lr = max(current_lr * self.factor, self.min_lr)
                new_lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                    initial_learning_rate=new_lr,
                    decay_steps=1000,
                    decay_rate=0.9,
                    staircase=True
                )
                self.model.optimizer.learning_rate = new_lr_schedule
                print(f'\nEpoch {epoch+1}: ReduceLROnPlateau reducing learning rate to {new_lr}.')
                self.wait = 0
                self.model.set_weights(self.best_weights)

# Compile and train
initial_learning_rate = 1e-3
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=1000, decay_rate=0.9, staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Multi-GPU Strategy
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = create_optimized_hybrid_model(img_height, img_width, len(numeric_columns))
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc'), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )

# Callbacks
reduce_lr = CustomReduceLROnPlateau(monitor='val_auc', factor=0.2, patience=5, min_lr=1e-6, mode='max')
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_auc', mode='max')
lr_logger = LearningRateLogger()

# Training
history = model.fit(
    train_dataset,
    epochs=100,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_dataset,
    validation_steps=validation_steps,
    callbacks=[reduce_lr, model_checkpoint, lr_logger]
)

# Save model
model.save('hybrid_model_optimized_final_fake.h5')

# Evaluation
val_loss, val_accuracy, val_auc, val_precision, val_recall = model.evaluate(val_dataset, steps=validation_steps)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation AUC: {val_auc:.4f}")
print(f"Validation Precision: {val_precision:.4f}")
print(f"Validation Recall: {val_recall:.4f}")

# Predictions for the validation dataset
y_pred = model.predict(val_dataset, steps=validation_steps)
y_pred_classes = (y_pred > 0.5).astype("int32")
y_true = np.concatenate([y for _, y in val_dataset.take(validation_steps)], axis=0)

# Confusion matrix and classification report
conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_report = classification_report(y_true, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

# Visualization of training history
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(history.history['auc'], label='Training AUC')
plt.plot(history.history['val_auc'], label='Validation AUC')
plt.title('Model AUC')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.legend()

plt.tight_layout()
plt.savefig('training_history.png')
plt.close()

# Visualization of learning rate
plt.figure(figsize=(10, 5))
plt.plot(history.history['lr'])
plt.title('Learning Rate over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.yscale('log')
plt.savefig('learning_rate.png')
plt.close()

# Visualization of confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig('confusion_matrix.png')
plt.close()

print("Visualizations have been saved as PNG files.")

# Save predictions and true labels
predictions_df = pd.DataFrame({
    'True_Label': y_true,
    'Predicted_Probability': y_pred.flatten(),
    'Predicted_Class': y_pred_classes.flatten()
})
predictions_df.to_csv('predictions.csv', index=False)
print("Predictions have been saved in 'predictions.csv'.")

# Model summary
model.summary()

# Save model architecture as image
tf.keras.utils.plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)
print("Model architecture has been saved as 'model_architecture.png'.")

# Calculate and output additional metrics
# ROC AUC
roc_auc = roc_auc_score(y_true, y_pred)
print(f"ROC AUC: {roc_auc:.4f}")

# Precision-Recall AUC
pr_auc = average_precision_score(y_true, y_pred)
print(f"Precision-Recall AUC: {pr_auc:.4f}")

# F1-Score at various thresholds
thresholds = np.arange(0, 1.01, 0.01)
f1_scores = [f1_score(y_true, y_pred > threshold) for threshold in thresholds]
best_threshold = thresholds[np.argmax(f1_scores)]
best_f1 = max(f1_scores)
print(f"Best F1-Score: {best_f1:.4f} at threshold {best_threshold:.2f}")

# Visualization of Precision-Recall curve
precision, recall, _ = precision_recall_curve(y_true, y_pred)
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.')
plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.savefig('precision_recall_curve.png')
plt.close()
print("Precision-Recall curve has been saved as 'precision_recall_curve.png'.")

# Visualization of ROC curve
fpr, tpr, _ = roc_curve(y_true, y_pred)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.savefig('roc_curve.png')
plt.close()
print("ROC curve has been saved as 'roc_curve.png'.")

# Save training history
with open('training_history.txt', 'w') as f:
    for key, values in history.history.items():
        f.write(f"{key}: {values}\n")
print("Training history has been saved in 'training_history.txt'.")

# Analyze misclassified examples
misclassified = np.where(y_pred_classes.flatten() != y_true)[0]
print(f"\nNumber of misclassified examples: {len(misclassified)}")

if len(misclassified) > 0:
    print("\nDetails of some misclassified examples:")
    for i in misclassified[:5]:  # Show details for the first 5 misclassified examples
        true_label = y_true[i]
        pred_label = y_pred_classes[i][0]
        pred_prob = y_pred[i][0]
        print(f"Example {i}: True Label: {true_label}, Predicted Label: {pred_label}, Predicted Probability: {pred_prob:.4f}")

# Model performance across different thresholds
accuracies = [accuracy_score(y_true, y_pred > threshold) for threshold in thresholds]

plt.figure(figsize=(10, 6))
plt.plot(thresholds, accuracies, label='Accuracy')
plt.plot(thresholds, f1_scores, label='F1-Score')
plt.axvline(x=0.5, color='r', linestyle='--', label='Default Threshold (0.5)')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Model Performance vs. Classification Threshold')
plt.legend()
plt.savefig('performance_vs_threshold.png')
plt.close()
print("Performance vs. Threshold plot has been saved as 'performance_vs_threshold.png'.")

# Optional: SHAP Analysis
try:
    import shap
    
    # Select a subset of validation data for SHAP analysis
    num_explain = 100
    explain_dataset = val_dataset.take(num_explain).unbatch()
    background_dataset = train_dataset.take(num_explain).unbatch()
    
    # Create an Explainer
    explainer = shap.DeepExplainer(model, background_dataset)
    
    # Calculate SHAP values
    shap_values = explainer.shap_values(explain_dataset)
    
    # Visualize SHAP values
    shap.summary_plot(shap_values[0], plot_type="bar", feature_names=numeric_columns, show=False)
    plt.savefig('shap_summary.png')
    plt.close()
    print("SHAP summary has been saved as 'shap_summary.png'.")
except ImportError:
    print("SHAP is not installed. Skipping SHAP analysis.")

print("\nAll analyses and visualizations are complete.")

# Summary of key metrics
print("\nSummary of key metrics:")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation AUC: {val_auc:.4f}")
print(f"Validation Precision: {val_precision:.4f}")
print(f"Validation Recall: {val_recall:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"Precision-Recall AUC: {pr_auc:.4f}")
print(f"Best F1-Score: {best_f1:.4f} at threshold {best_threshold:.2f}")

# Save summary to a text file
with open('model_summary.txt', 'w') as f:
    f.write("Summary of key metrics:\n")
    f.write(f"Validation Accuracy: {val_accuracy:.4f}\n")
    f.write(f"Validation AUC: {val_auc:.4f}\n")
    f.write(f"Validation Precision: {val_precision:.4f}\n")
    f.write(f"Validation Recall: {val_recall:.4f}\n")
    f.write(f"ROC AUC: {roc_auc:.4f}\n")
    f.write(f"Precision-Recall AUC: {pr_auc:.4f}\n")
    f.write(f"Best F1-Score: {best_f1:.4f} at threshold {best_threshold:.2f}\n")

print("\nA summary of key metrics has been saved in 'model_summary.txt'.")
print("\nThe script has been executed successfully. All results and visualizations have been saved.")



Original CSV data loaded. Number of rows: 15435
Original distribution of labels:
label_majority
2    9758
1    3241
3    2113
4     323
Name: count, dtype: int64
1 Physical GPUs, 1 Logical GPUs
Mixed precision policy set to mixed_float16
Found images: Filtered: 646, Not filtered: 27994
Sample of extracted filenames:
0       00042065
1    00042065Alt
2       00043749
3    00043749Alt
4       00080967
Name: filename, dtype: object
After undersampling: Number of rows: 2323
Distribution of labels after undersampling:
label
0    2000
1     323
Name: count, dtype: int64
Training data: 1858, Validation data: 465
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Epoch 1/100
58/58 [==============================] - 21s 163ms/step - loss: 0.7933 - accuracy: 0.6067 - auc: 0.6359 - precision_4: 0.1887 - recall_4: 0.5543 - val_loss: 0.6030 - val_accuracy: 0.7232 - val_auc: 0.7177 - val_precision_4: 0.2701 - val_recall_4: 0.6066 - lr: 0.0010
Epoch 

AttributeError: module 'pydot' has no attribute 'InvocationException'

# hybrid_model_optimized_final.h5

In [3]:
                                                       ##### Asle jens ine ###########




import tensorflow as tf
from tensorflow.keras import layers, models, Input, mixed_precision
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, accuracy_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

# Load the DataFrame
file_path = r"C:\Users\meh91075\Downloads\AutoSplit_all_in_one_dataset_05112024.csv"
#"C:\Users\meh91075\Downloads\merged_features_filtered.csv"
df = pd.read_csv(file_path)

# Entferne label_standard
df = df.drop('label_standard', axis=1)


print(f"Original CSV data loaded. Number of rows: {len(df)}")
print(f"Original distribution of labels:")
print(df['label_majority'].value_counts())

# Optimize GPU configuration
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        print(e)

# Activate Mixed Precision
if tf.test.is_built_with_cuda():
    policy = mixed_precision.Policy('mixed_float16')
    mixed_precision.set_global_policy(policy)
    print("Mixed precision policy set to mixed_float16")
else:
    print("CUDA not available, using default precision")




# Pfade und Konfiguration
base_dir =  r"C:\Users\meh91075\Desktop\DataSet(S1)"
filtered_dir = os.path.join(base_dir, 'New_filtered_data_label0')
non_filtered_dir = os.path.join(base_dir, 'New_non_filtered_data')



img_height, img_width = 224, 224
batch_size = 32

def get_image_paths(directory):
    return [os.path.join(root, file) for root, _, files in os.walk(directory) 
            for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

filtered_images = get_image_paths(filtered_dir)
non_filtered_images = get_image_paths(non_filtered_dir)
print(f"Found images: Filtered: {len(filtered_images)}, Not filtered: {len(non_filtered_images)}")

def extract_filename(path):
    basename = os.path.basename(path)
    return basename.replace('_ISO', '').replace('_ISOAlt', '').split('.')[0]

image_df = pd.DataFrame({
    'image_path': filtered_images + non_filtered_images,
    'label': [1] * len(filtered_images) + [0] * len(non_filtered_images),
    'filename': [extract_filename(path) for path in filtered_images + non_filtered_images]
})

print("Sample of extracted filenames:")
print(image_df['filename'].head())

# Adjusted merge process
merged_df = pd.merge(df, image_df[['filename', 'image_path']], on='filename', how='left')
merged_df['label'] = merged_df['label_majority'].map({1: 0, 2: 0, 3: 0, 4: 1})
merged_df['has_image'] = merged_df['image_path'].notna()

# Filter only entries with associated images
merged_df_with_images = merged_df[merged_df['has_image']]

# Undersampling of the majority class
majority_class = merged_df_with_images[merged_df_with_images['label'] == 0]
minority_class = merged_df_with_images[merged_df_with_images['label'] == 1]

majority_downsampled = majority_class.sample(n=2000, random_state=42)
balanced_df = pd.concat([majority_downsampled, minority_class])

print(f"After undersampling: Number of rows: {len(balanced_df)}")
print("Distribution of labels after undersampling:")
print(balanced_df['label'].value_counts())

# Train-Test-Split
train_df, val_df = train_test_split(balanced_df, test_size=0.2, stratify=balanced_df['label'], random_state=42)
print(f"Training data: {len(train_df)}, Validation data: {len(val_df)}")

# Determine the numeric columns
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_columns = [col for col in numeric_columns if col not in ['label_majority', 'label']]

# Normalization of numeric data
scaler = StandardScaler()
train_df[numeric_columns] = scaler.fit_transform(train_df[numeric_columns])
val_df[numeric_columns] = scaler.transform(val_df[numeric_columns])

# Optimized data generator function with data augmentation
def create_dataset(dataframe, is_training=True):
    def parse_function(filename, label, *numeric_data):
        img = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [img_height, img_width])
        img = tf.keras.applications.resnet_v2.preprocess_input(img)
        
        if is_training:
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_brightness(img, max_delta=0.2)
            img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
            img = tf.image.random_saturation(img, lower=0.8, upper=1.2)
            img = tf.image.random_hue(img, max_delta=0.1)
        
        numeric_data = tf.convert_to_tensor(numeric_data, dtype=tf.float32)
        numeric_data = tf.squeeze(numeric_data)
        return (img, numeric_data), label

    dataset = tf.data.Dataset.from_tensor_slices((
        dataframe['image_path'].values,
        dataframe['label'].values,
        dataframe[numeric_columns].values
    ))
    
    dataset = dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    
    if is_training:
        dataset = dataset.shuffle(buffer_size=len(dataframe)).repeat()
    else:
        dataset = dataset.repeat()
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

# Create the datasets
train_dataset = create_dataset(train_df)
val_dataset = create_dataset(val_df, is_training=False)

# Calculate steps per epoch
steps_per_epoch = len(train_df) // batch_size
validation_steps = len(val_df) // batch_size

# Optimized model definition
def create_optimized_hybrid_model(img_height, img_width, num_numeric_features):
    # Image processing branch (CNN)
    img_input = Input(shape=(img_height, img_width, 3))
    
    # Use a pre-trained model for feature extraction
    base_model = tf.keras.applications.ResNet50V2(include_top=False, weights='imagenet', input_tensor=img_input)
    
    # Freeze the pre-trained layers
    base_model.trainable = False
    
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    # Numerical data branch (MLP)
    num_input = Input(shape=(num_numeric_features,))
    y = layers.Dense(128, activation='relu')(num_input)
    y = layers.BatchNormalization()(y)
    y = layers.Dropout(0.3)(y)
    y = layers.Dense(256, activation='relu')(y)
    y = layers.BatchNormalization()(y)
    y = layers.Dropout(0.4)(y)
    y = layers.Dense(128, activation='relu')(y)
    y = layers.BatchNormalization()(y)
    y = layers.Dropout(0.3)(y)
    
    # Combining the branches
    combined = layers.concatenate([x, y])
    
    # Final layers
    z = layers.Dense(256, activation='relu')(combined)
    z = layers.BatchNormalization()(z)
    z = layers.Dropout(0.5)(z)
    z = layers.Dense(128, activation='relu')(z)
    z = layers.BatchNormalization()(z)
    z = layers.Dropout(0.5)(z)
    
    # Output layer
    output = layers.Dense(1, activation='sigmoid')(z)
    
    model = models.Model(inputs=[img_input, num_input], outputs=output)
    return model

# Learning Rate Logger Callback
class LearningRateLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            lr = self.model.optimizer.learning_rate
            if isinstance(lr, tf.keras.optimizers.schedules.LearningRateSchedule):
                lr = lr(self.model.optimizer.iterations)
            logs['lr'] = tf.keras.backend.get_value(lr)

# Custom ReduceLROnPlateau Callback
class CustomReduceLROnPlateau(tf.keras.callbacks.Callback):
    def __init__(self, factor=0.1, patience=10, min_lr=0, monitor='val_loss', mode='min'):
        super(CustomReduceLROnPlateau, self).__init__()
        self.factor = factor
        self.patience = patience
        self.min_lr = min_lr
        self.monitor = monitor
        self.mode = mode
        self.best = np.Inf if mode == 'min' else -np.Inf
        self.wait = 0
        self.best_weights = None

    def on_train_begin(self, logs=None):
        self.best = np.Inf if self.mode == 'min' else -np.Inf
        self.wait = 0
        self.best_weights = None

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        current = logs.get(self.monitor)
        if current is None:
            return
        
        if self.mode == 'min' and current < self.best:
            self.best = current
            self.wait = 0
            self.best_weights = self.model.get_weights()
        elif self.mode == 'max' and current > self.best:
            self.best = current
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
        
        if self.wait >= self.patience:
            current_lr = self.model.optimizer.learning_rate
            if isinstance(current_lr, tf.keras.optimizers.schedules.LearningRateSchedule):
                current_lr = current_lr(self.model.optimizer.iterations)
            current_lr = float(current_lr.numpy())
            
            if current_lr > self.min_lr:
                new_lr = max(current_lr * self.factor, self.min_lr)
                new_lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                    initial_learning_rate=new_lr,
                    decay_steps=1000,
                    decay_rate=0.9,
                    staircase=True
                )
                self.model.optimizer.learning_rate = new_lr_schedule
                print(f'\nEpoch {epoch+1}: ReduceLROnPlateau reducing learning rate to {new_lr}.')
                self.wait = 0
                self.model.set_weights(self.best_weights)

# Compile and train
initial_learning_rate = 1e-3
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=1000, decay_rate=0.9, staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Multi-GPU Strategy
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = create_optimized_hybrid_model(img_height, img_width, len(numeric_columns))
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc'), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )

# Callbacks
reduce_lr = CustomReduceLROnPlateau(monitor='val_auc', factor=0.2, patience=5, min_lr=1e-6, mode='max')
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_auc', mode='max')
lr_logger = LearningRateLogger()

# Training
history = model.fit(
    train_dataset,
    epochs=100,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_dataset,
    validation_steps=validation_steps,
    callbacks=[reduce_lr, model_checkpoint, lr_logger]
)

# Save model
model.save('hybrid_model_optimized_final.h5')

# Evaluation
val_loss, val_accuracy, val_auc, val_precision, val_recall = model.evaluate(val_dataset, steps=validation_steps)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation AUC: {val_auc:.4f}")
print(f"Validation Precision: {val_precision:.4f}")
print(f"Validation Recall: {val_recall:.4f}")

# Predictions for the validation dataset
y_pred = model.predict(val_dataset, steps=validation_steps)
y_pred_classes = (y_pred > 0.5).astype("int32")
y_true = np.concatenate([y for _, y in val_dataset.take(validation_steps)], axis=0)

# Confusion matrix and classification report
conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_report = classification_report(y_true, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

# Visualization of training history
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(history.history['auc'], label='Training AUC')
plt.plot(history.history['val_auc'], label='Validation AUC')
plt.title('Model AUC')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.legend()

plt.tight_layout()
plt.savefig('training_history.png')
plt.close()

# Visualization of learning rate
plt.figure(figsize=(10, 5))
plt.plot(history.history['lr'])
plt.title('Learning Rate over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.yscale('log')
plt.savefig('learning_rate.png')
plt.close()

# Visualization of confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig('confusion_matrix.png')
plt.close()

print("Visualizations have been saved as PNG files.")

# Save predictions and true labels
predictions_df = pd.DataFrame({
    'True_Label': y_true,
    'Predicted_Probability': y_pred.flatten(),
    'Predicted_Class': y_pred_classes.flatten()
})
predictions_df.to_csv('predictions.csv', index=False)
print("Predictions have been saved in 'predictions.csv'.")

# Model summary
model.summary()

# Save model architecture as image
tf.keras.utils.plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)
print("Model architecture has been saved as 'model_architecture.png'.")

# Calculate and output additional metrics
# ROC AUC
roc_auc = roc_auc_score(y_true, y_pred)
print(f"ROC AUC: {roc_auc:.4f}")

# Precision-Recall AUC
pr_auc = average_precision_score(y_true, y_pred)
print(f"Precision-Recall AUC: {pr_auc:.4f}")

# F1-Score at various thresholds
thresholds = np.arange(0, 1.01, 0.01)
f1_scores = [f1_score(y_true, y_pred > threshold) for threshold in thresholds]
best_threshold = thresholds[np.argmax(f1_scores)]
best_f1 = max(f1_scores)
print(f"Best F1-Score: {best_f1:.4f} at threshold {best_threshold:.2f}")

# Visualization of Precision-Recall curve
precision, recall, _ = precision_recall_curve(y_true, y_pred)
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.')
plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.savefig('precision_recall_curve.png')
plt.close()
print("Precision-Recall curve has been saved as 'precision_recall_curve.png'.")

# Visualization of ROC curve
fpr, tpr, _ = roc_curve(y_true, y_pred)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.savefig('roc_curve.png')
plt.close()
print("ROC curve has been saved as 'roc_curve.png'.")

# Save training history
with open('training_history.txt', 'w') as f:
    for key, values in history.history.items():
        f.write(f"{key}: {values}\n")
print("Training history has been saved in 'training_history.txt'.")

# Analyze misclassified examples
misclassified = np.where(y_pred_classes.flatten() != y_true)[0]
print(f"\nNumber of misclassified examples: {len(misclassified)}")

if len(misclassified) > 0:
    print("\nDetails of some misclassified examples:")
    for i in misclassified[:5]:  # Show details for the first 5 misclassified examples
        true_label = y_true[i]
        pred_label = y_pred_classes[i][0]
        pred_prob = y_pred[i][0]
        print(f"Example {i}: True Label: {true_label}, Predicted Label: {pred_label}, Predicted Probability: {pred_prob:.4f}")

# Model performance across different thresholds
accuracies = [accuracy_score(y_true, y_pred > threshold) for threshold in thresholds]

plt.figure(figsize=(10, 6))
plt.plot(thresholds, accuracies, label='Accuracy')
plt.plot(thresholds, f1_scores, label='F1-Score')
plt.axvline(x=0.5, color='r', linestyle='--', label='Default Threshold (0.5)')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Model Performance vs. Classification Threshold')
plt.legend()
plt.savefig('performance_vs_threshold.png')
plt.close()
print("Performance vs. Threshold plot has been saved as 'performance_vs_threshold.png'.")

# Optional: SHAP Analysis
try:
    import shap
    
    # Select a subset of validation data for SHAP analysis
    num_explain = 100
    explain_dataset = val_dataset.take(num_explain).unbatch()
    background_dataset = train_dataset.take(num_explain).unbatch()
    
    # Create an Explainer
    explainer = shap.DeepExplainer(model, background_dataset)
    
    # Calculate SHAP values
    shap_values = explainer.shap_values(explain_dataset)
    
    # Visualize SHAP values
    shap.summary_plot(shap_values[0], plot_type="bar", feature_names=numeric_columns, show=False)
    plt.savefig('shap_summary.png')
    plt.close()
    print("SHAP summary has been saved as 'shap_summary.png'.")
except ImportError:
    print("SHAP is not installed. Skipping SHAP analysis.")

print("\nAll analyses and visualizations are complete.")

# Summary of key metrics
print("\nSummary of key metrics:")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation AUC: {val_auc:.4f}")
print(f"Validation Precision: {val_precision:.4f}")
print(f"Validation Recall: {val_recall:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"Precision-Recall AUC: {pr_auc:.4f}")
print(f"Best F1-Score: {best_f1:.4f} at threshold {best_threshold:.2f}")

# Save summary to a text file
with open('model_summary.txt', 'w') as f:
    f.write("Summary of key metrics:\n")
    f.write(f"Validation Accuracy: {val_accuracy:.4f}\n")
    f.write(f"Validation AUC: {val_auc:.4f}\n")
    f.write(f"Validation Precision: {val_precision:.4f}\n")
    f.write(f"Validation Recall: {val_recall:.4f}\n")
    f.write(f"ROC AUC: {roc_auc:.4f}\n")
    f.write(f"Precision-Recall AUC: {pr_auc:.4f}\n")
    f.write(f"Best F1-Score: {best_f1:.4f} at threshold {best_threshold:.2f}\n")

print("\nA summary of key metrics has been saved in 'model_summary.txt'.")
print("\nThe script has been executed successfully. All results and visualizations have been saved.")



Original CSV data loaded. Number of rows: 19274
Original distribution of labels:
label_majority
2    10850
1     3485
4     2745
3     2194
Name: count, dtype: int64
1 Physical GPUs, 1 Logical GPUs
Mixed precision policy set to mixed_float16
Found images: Filtered: 5664, Not filtered: 27994
Sample of extracted filenames:
0       00042065
1    00042065Alt
2       00043749
3    00043749Alt
4       00080967
Name: filename, dtype: object
After undersampling: Number of rows: 4499
Distribution of labels after undersampling:
label
1    2499
0    2000
Name: count, dtype: int64
Training data: 3599, Validation data: 900
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Epoch 1/100
112/112 [==============================] - 28s 132ms/step - loss: 0.6059 - accuracy: 0.7266 - auc: 0.8002 - precision_2: 0.7569 - recall_2: 0.7477 - val_loss: 0.4200 - val_accuracy: 0.8304 - val_auc: 0.8883 - val_precision_2: 0.8046 - val_recall_2: 0.9177 - lr: 0.0010

AttributeError: module 'pydot' has no attribute 'InvocationException'

## hybrid_model_optimized1.h5

In [30]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input, mixed_precision
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, accuracy_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

# Load the DataFrame
file_path = r"C:\Users\meh91075\Downloads\AutoSplit_dataset_09092024.csv"
#C:\Users\meh91075\Downloads\merged_features_filtered.csv"
df = pd.read_csv(file_path)
print(f"Original CSV data loaded. Number of rows: {len(df)}")
print(f"Original distribution of labels:")
print(df['label_majority'].value_counts())

# Optimize GPU configuration
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        print(e)

# Activate Mixed Precision
if tf.test.is_built_with_cuda():
    policy = mixed_precision.Policy('mixed_float16')
    mixed_precision.set_global_policy(policy)
    print("Mixed precision policy set to mixed_float16")
else:
    print("CUDA not available, using default precision")


# Pfade und Konfiguration
base_dir = "C:/Users/meh91075/Desktop/02_AutoSplit/02_AutoSplit/10_Datasets/01_Autosplit_dataset"
filtered_dir = os.path.join(base_dir, 'New_filtered_data_label4')
non_filtered_dir = os.path.join(base_dir, 'New_non_filtered_data')

# # Paths and Configuration
# base_dir = "C:/Users/meh91075/Desktop/02_AutoSplit/02_AutoSplit/10_Datasets/01_Autosplit_dataset"
# base_dir1 = "C:/Users/meh91075/Downloads/04_Standardparts"
# filtered_dir = os.path.join(base_dir1, 'png')
# non_filtered_dir = os.path.join(base_dir, 'New_non_filtered_data')

img_height, img_width = 224, 224
batch_size = 32

def get_image_paths(directory):
    return [os.path.join(root, file) for root, _, files in os.walk(directory) 
            for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

filtered_images = get_image_paths(filtered_dir)
non_filtered_images = get_image_paths(non_filtered_dir)
print(f"Found images: Filtered: {len(filtered_images)}, Not filtered: {len(non_filtered_images)}")

def extract_filename(path):
    basename = os.path.basename(path)
    return basename.replace('_ISO', '').replace('_ISOAlt', '').split('.')[0]

image_df = pd.DataFrame({
    'image_path': filtered_images + non_filtered_images,
    'label': [1] * len(filtered_images) + [0] * len(non_filtered_images),
    'filename': [extract_filename(path) for path in filtered_images + non_filtered_images]
})

print("Sample of extracted filenames:")
print(image_df['filename'].head())

# Adjusted merge process
merged_df = pd.merge(df, image_df[['filename', 'image_path']], on='filename', how='left')
merged_df['label'] = merged_df['label_majority'].map({1: 0, 2: 0, 3: 0, 4: 1})
merged_df['has_image'] = merged_df['image_path'].notna()

# Filter only entries with associated images
merged_df_with_images = merged_df[merged_df['has_image']]

# Undersampling of the majority class
majority_class = merged_df_with_images[merged_df_with_images['label'] == 0]
minority_class = merged_df_with_images[merged_df_with_images['label'] == 1]

majority_downsampled = majority_class.sample(n=2000, random_state=42)
balanced_df = pd.concat([majority_downsampled, minority_class])

print(f"After undersampling: Number of rows: {len(balanced_df)}")
print("Distribution of labels after undersampling:")
print(balanced_df['label'].value_counts())

# Train-Test-Split
train_df, val_df = train_test_split(balanced_df, test_size=0.2, stratify=balanced_df['label'], random_state=42)
print(f"Training data: {len(train_df)}, Validation data: {len(val_df)}")

# Determine the numeric columns
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_columns = [col for col in numeric_columns if col not in ['label_majority', 'label']]

# Normalization of numeric data
scaler = StandardScaler()
train_df[numeric_columns] = scaler.fit_transform(train_df[numeric_columns])
val_df[numeric_columns] = scaler.transform(val_df[numeric_columns])

# Optimized data generator function with data augmentation
def create_dataset(dataframe, is_training=True):
    def parse_function(filename, label, *numeric_data):
        img = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [img_height, img_width])
        img = tf.cast(img, tf.float32) / 255.0
        if is_training:
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_brightness(img, max_delta=0.2)
            img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
            img = tf.image.random_saturation(img, lower=0.8, upper=1.2)
            img = tf.image.random_hue(img, max_delta=0.1)
        
        numeric_data = tf.convert_to_tensor(numeric_data, dtype=tf.float32)
        numeric_data = tf.squeeze(numeric_data)
        return (img, numeric_data), label

    dataset = tf.data.Dataset.from_tensor_slices((
        dataframe['image_path'].values,
        dataframe['label'].values,
        dataframe[numeric_columns].values
    ))
    
    dataset = dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    
    if is_training:
        dataset = dataset.shuffle(buffer_size=len(dataframe)).repeat()
    else:
        dataset = dataset.repeat()
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

# Create the datasets
train_dataset = create_dataset(train_df)
val_dataset = create_dataset(val_df, is_training=False)

# Calculate steps per epoch
steps_per_epoch = len(train_df) // batch_size
validation_steps = len(val_df) // batch_size

# Updated optimized model definition
def create_optimized_model(img_height, img_width, num_numeric_features):
    # Image processing branch (CNN)
    img_input = Input(shape=(img_height, img_width, 3))
    
    # 1Blue-1Orange
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(img_input)
    x = layers.MaxPooling2D()(x)
    
    # 1Blue-1Orange
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    
    # 1Blue-1Orange
    x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    
    # 1Blue-1Orange
    x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    
    # 1Yellow (Flatten)
    x = layers.Flatten()(x)
    
    # 1Green
    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    # 1Green
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    # Numerical data branch (MLP)
    num_input = Input(shape=(num_numeric_features,))
    y = layers.Dense(64, activation='relu')(num_input)
    y = layers.BatchNormalization()(y)
    y = layers.Dropout(0.3)(y)
    
    # Combining the branches
    combined = layers.concatenate([x, y])
    
    # 1 fully connected Green
    z = layers.Dense(128, activation='relu')(combined)
    z = layers.BatchNormalization()(z)
    z = layers.Dropout(0.5)(z)
    
    # Output layer
    output = layers.Dense(1, activation='sigmoid')(z)
    
    model = models.Model(inputs=[img_input, num_input], outputs=output)
    return model

# Learning Rate Logger Callback
class LearningRateLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            lr = self.model.optimizer.learning_rate
            if isinstance(lr, tf.keras.optimizers.schedules.LearningRateSchedule):
                lr = lr(self.model.optimizer.iterations)
            logs['lr'] = tf.keras.backend.get_value(lr)

# Custom ReduceLROnPlateau Callback
class CustomReduceLROnPlateau(tf.keras.callbacks.Callback):
    def __init__(self, factor=0.1, patience=10, min_lr=0, monitor='val_loss', mode='min'):
        super(CustomReduceLROnPlateau, self).__init__()
        self.factor = factor
        self.patience = patience
        self.min_lr = min_lr
        self.monitor = monitor
        self.mode = mode
        self.best = np.Inf if mode == 'min' else -np.Inf
        self.wait = 0
        self.best_weights = None

    def on_train_begin(self, logs=None):
        self.best = np.Inf if self.mode == 'min' else -np.Inf
        self.wait = 0
        self.best_weights = None

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        current = logs.get(self.monitor)
        if current is None:
            return
        
        if self.mode == 'min' and current < self.best:
            self.best = current
            self.wait = 0
            self.best_weights = self.model.get_weights()
        elif self.mode == 'max' and current > self.best:
            self.best = current
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
        
        if self.wait >= self.patience:
            current_lr = self.model.optimizer.learning_rate
            if isinstance(current_lr, tf.keras.optimizers.schedules.LearningRateSchedule):
                current_lr = current_lr(self.model.optimizer.iterations)
            current_lr = float(current_lr.numpy())
            
            if current_lr > self.min_lr:
                new_lr = max(current_lr * self.factor, self.min_lr)
                new_lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                    initial_learning_rate=new_lr,
                    decay_steps=1000,
                    decay_rate=0.9,
                    staircase=True
                )
                self.model.optimizer.learning_rate = new_lr_schedule
                print(f'\nEpoch {epoch+1}: ReduceLROnPlateau reducing learning rate to {new_lr}.')
                self.wait = 0
                self.model.set_weights(self.best_weights)

# Compile and train
initial_learning_rate = 1e-3
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=1000, decay_rate=0.9, staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Multi-GPU Strategy
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = create_optimized_model(img_height, img_width, len(numeric_columns))
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc_1'), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )

# Callbacks
reduce_lr = CustomReduceLROnPlateau(monitor='val_auc_1', factor=0.2, patience=5, min_lr=1e-6, mode='max')
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_auc_1', mode='max')
lr_logger = LearningRateLogger()

# Training
history = model.fit(
    train_dataset,
    epochs=100,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_dataset,
    validation_steps=validation_steps,
    callbacks=[reduce_lr, model_checkpoint, lr_logger]
)

# Save model
model.save('hybrid_model_optimized1.h5')

# Evaluation
val_loss, val_accuracy, val_auc, val_precision, val_recall = model.evaluate(val_dataset, steps=validation_steps)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation AUC: {val_auc:.4f}")
print(f"Validation Precision: {val_precision:.4f}")
print(f"Validation Recall: {val_recall:.4f}")

# Predictions for the validation dataset
y_pred = model.predict(val_dataset, steps=validation_steps)
y_pred_classes = (y_pred > 0.5).astype("int32")
y_true = np.concatenate([y for _, y in val_dataset.take(validation_steps)], axis=0)

# Confusion matrix and classification report
conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_report = classification_report(y_true, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

# Visualization of training history (continued)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(history.history['auc_1'], label='Training AUC')
plt.plot(history.history['val_auc_1'], label='Validation AUC')
plt.title('Model AUC')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.legend()

plt.tight_layout()
plt.savefig('training_history.png')
plt.close()

# Visualization of learning rate
plt.figure(figsize=(10, 5))
plt.plot(history.history['lr'])
plt.title('Learning Rate over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.yscale('log')
plt.savefig('learning_rate.png')
plt.close()

# Visualization of confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig('confusion_matrix.png')
plt.close()

print("Visualizations have been saved as PNG files.")

# Save predictions and true labels
predictions_df = pd.DataFrame({
    'True_Label': y_true,
    'Predicted_Probability': y_pred.flatten(),
    'Predicted_Class': y_pred_classes.flatten()
})
predictions_df.to_csv('predictions.csv', index=False)
print("Predictions have been saved in 'predictions.csv'.")

# Model summary
model.summary()

# Save model architecture as image
tf.keras.utils.plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)
print("Model architecture has been saved as 'model_architecture.png'.")

# Calculate and output additional metrics
# ROC AUC
roc_auc = roc_auc_score(y_true, y_pred)
print(f"ROC AUC: {roc_auc:.4f}")

# Precision-Recall AUC
pr_auc = average_precision_score(y_true, y_pred)
print(f"Precision-Recall AUC: {pr_auc:.4f}")

# F1-Score at various thresholds
thresholds = np.arange(0, 1.01, 0.01)
f1_scores = [f1_score(y_true, y_pred > threshold) for threshold in thresholds]
best_threshold = thresholds[np.argmax(f1_scores)]
best_f1 = max(f1_scores)
print(f"Best F1-Score: {best_f1:.4f} at threshold {best_threshold:.2f}")

# Visualization of Precision-Recall curve
precision, recall, _ = precision_recall_curve(y_true, y_pred)
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.')
plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.savefig('precision_recall_curve.png')
plt.close()
print("Precision-Recall curve has been saved as 'precision_recall_curve.png'.")

# Save training history
with open('training_history.txt', 'w') as f:
    for key, values in history.history.items():
        f.write(f"{key}: {values}\n")
print("Training history has been saved in 'training_history.txt'.")

print("Model training and evaluation completed.")

# Visualization of ROC curve
fpr, tpr, _ = roc_curve(y_true, y_pred)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.savefig('roc_curve.png')
plt.close()
print("ROC curve has been saved as 'roc_curve.png'.")

# Analyze misclassified examples
misclassified = np.where(y_pred_classes.flatten() != y_true)[0]
print(f"\nNumber of misclassified examples: {len(misclassified)}")

if len(misclassified) > 0:
    print("\nDetails of some misclassified examples:")
    for i in misclassified[:5]:  # Show details for the first 5 misclassified examples
        true_label = y_true[i]
        pred_label = y_pred_classes[i][0]
        pred_prob = y_pred[i][0]
        print(f"Example {i}: True Label: {true_label}, Predicted Label: {pred_label}, Predicted Probability: {pred_prob:.4f}")

# Model performance across different thresholds
accuracies = [accuracy_score(y_true, y_pred > threshold) for threshold in thresholds]

plt.figure(figsize=(10, 6))
plt.plot(thresholds, accuracies, label='Accuracy')
plt.plot(thresholds, f1_scores, label='F1-Score')
plt.axvline(x=0.5, color='r', linestyle='--', label='Default Threshold (0.5)')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Model Performance vs. Classification Threshold')
plt.legend()
plt.savefig('performance_vs_threshold.png')
plt.close()
print("Performance vs. Threshold plot has been saved as 'performance_vs_threshold.png'.")

# Optional: SHAP Analysis
try:
    import shap
    
    # Select a subset of validation data for SHAP analysis
    num_explain = 100
    explain_dataset = val_dataset.take(num_explain).unbatch()
    background_dataset = train_dataset.take(num_explain).unbatch()
    
    # Create an Explainer
    explainer = shap.DeepExplainer(model, background_dataset)
    
    # Calculate SHAP values
    shap_values = explainer.shap_values(explain_dataset)
    
    # Visualize SHAP values
    shap.summary_plot(shap_values[0], plot_type="bar", feature_names=numeric_columns, show=False)
    plt.savefig('shap_summary.png')
    plt.close()
    print("SHAP summary has been saved as 'shap_summary.png'.")
except ImportError:
    print("SHAP is not installed. Skipping SHAP analysis.")

print("\nAll analyses and visualizations are complete.")

# Summary of key metrics
print("\nSummary of key metrics:")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation AUC: {val_auc:.4f}")
print(f"Validation Precision: {val_precision:.4f}")
print(f"Validation Recall: {val_recall:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"Precision-Recall AUC: {pr_auc:.4f}")
print(f"Best F1-Score: {best_f1:.4f} at threshold {best_threshold:.2f}")

# Save summary to a text file
with open('model_summary.txt', 'w') as f:
    f.write("Summary of key metrics:\n")
    f.write(f"Validation Accuracy: {val_accuracy:.4f}\n")
    f.write(f"Validation AUC: {val_auc:.4f}\n")
    f.write(f"Validation Precision: {val_precision:.4f}\n")
    f.write(f"Validation Recall: {val_recall:.4f}\n")
    f.write(f"ROC AUC: {roc_auc:.4f}\n")
    f.write(f"Precision-Recall AUC: {pr_auc:.4f}\n")
    f.write(f"Best F1-Score: {best_f1:.4f} at threshold {best_threshold:.2f}\n")

print("\nA summary of key metrics has been saved in 'model_summary.txt'.")
print("\nThe script has been executed successfully. All results and visualizations have been saved.")

Original CSV data loaded. Number of rows: 17762
Original distribution of labels:
label_majority
2    9758
1    3241
4    2650
3    2113
Name: count, dtype: int64
1 Physical GPUs, 1 Logical GPUs
Mixed precision policy set to mixed_float16
Found images: Filtered: 5018, Not filtered: 27994
Sample of extracted filenames:
0       100218_83ec552f_0000_1
1    100218_83ec552f_0000_1Alt
2       100622_1c7038f5_0006_1
3    100622_1c7038f5_0006_1Alt
4       100622_1c7038f5_0013_1
Name: filename, dtype: object
After undersampling: Number of rows: 4509
Distribution of labels after undersampling:
label
1    2509
0    2000
Name: count, dtype: int64
Training data: 3607, Validation data: 902
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Epoch 1/100
112/112 [==============================] - 37s 217ms/step - loss: 0.8482 - accuracy: 0.5977 - auc_1: 0.6337 - precision_4: 0.6411 - recall_4: 0.6314 - val_loss: 0.8686 - val_accuracy: 0.4431 - val_auc_1

C:\Users\meh91075\AppData\Local\Temp\ipykernel_14680\2272295425.py:328: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


Visualizations have been saved as PNG files.
Predictions have been saved in 'predictions.csv'.
Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_17 (Conv2D)             (None, 224, 224, 64  1792        ['input_11[0][0]']               
                                )                                                                 
                                                                                                  
 max_pooling2d_17 (MaxPooling2D  (None, 112, 112, 64  0          ['conv2d_17[0][0]']            

AttributeError: module 'pydot' has no attribute 'InvocationException'

## final_model.h5

In [28]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input, mixed_precision
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, f1_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

# Konstanten
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
EPOCHS = 100

# GPU Konfiguration
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Mixed Precision
if tf.test.is_built_with_cuda():
    policy = mixed_precision.Policy('mixed_float16')
    mixed_precision.set_global_policy(policy)

def create_dataset(dataframe, numeric_columns, is_training=True):
    def parse_function(filename, label, numeric_data):
        # Bild verarbeiten
        img = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
        img = tf.cast(img, tf.float32) / 255.0
        
        if is_training:
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_brightness(img, max_delta=0.2)
            img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
        
        return (img, numeric_data), label

    # Dataset erstellen
    dataset = tf.data.Dataset.from_tensor_slices((
        dataframe['image_path'].values,
        dataframe['label'].values,
        dataframe[numeric_columns].values.astype(np.float32)
    ))
    
    dataset = dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    if is_training:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

def create_model(num_numeric_features):
    # Image processing branch
    img_input = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(img_input)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    
    x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    
    # Numeric data branch
    num_input = Input(shape=(num_numeric_features,))
    y = layers.Dense(64, activation='relu')(num_input)
    y = layers.BatchNormalization()(y)
    y = layers.Dropout(0.3)(y)
    
    # Combine branches
    combined = layers.concatenate([x, y])
    
    z = layers.Dense(128, activation='relu')(combined)
    z = layers.BatchNormalization()(z)
    z = layers.Dropout(0.5)(z)
    
    output = layers.Dense(1, activation='sigmoid')(z)
    
    return models.Model(inputs=[img_input, num_input], outputs=output)

def main():
    # Daten laden
    file_path = r"C:\Users\meh91075\Downloads\AutoSplit_dataset_09092024.csv"
    df = pd.read_csv(file_path)
    
    # Bilder laden
    filtered_dir = os.path.join("C:/Users/meh91075/Desktop/02_AutoSplit/02_AutoSplit/10_Datasets/01_Autosplit_dataset", 'New_filtered_data_label4')
    non_filtered_dir = os.path.join("C:/Users/meh91075/Desktop/02_AutoSplit/02_AutoSplit/10_Datasets/01_Autosplit_dataset", 'New_non_filtered_data')
    
    def get_image_paths(directory):
        return [os.path.join(root, file) for root, _, files in os.walk(directory) 
                for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

    filtered_images = get_image_paths(filtered_dir)
    non_filtered_images = get_image_paths(non_filtered_dir)
    
    def extract_filename(path):
        return os.path.basename(path).replace('_ISO', '').replace('_ISOAlt', '').split('.')[0]

    image_df = pd.DataFrame({
        'image_path': filtered_images + non_filtered_images,
        'label': [1] * len(filtered_images) + [0] * len(non_filtered_images),
        'filename': [extract_filename(path) for path in filtered_images + non_filtered_images]
    })
    
    # Daten vorbereiten
    merged_df = pd.merge(df, image_df[['filename', 'image_path']], on='filename', how='left')
    merged_df['label'] = merged_df['label_majority'].map({1: 0, 2: 0, 3: 0, 4: 1})
    merged_df = merged_df[merged_df['image_path'].notna()]
    
    # Daten balancieren
    minority_class = merged_df[merged_df['label'] == 1]
    majority_class = merged_df[merged_df['label'] == 0].sample(n=len(minority_class), random_state=42)
    balanced_df = pd.concat([majority_class, minority_class])
    
    # Train-Test-Split
    train_df, val_df = train_test_split(balanced_df, test_size=0.2, stratify=balanced_df['label'], random_state=42)
    
    # Numerische Features
    numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
    numeric_columns = [col for col in numeric_columns if col not in ['label_majority', 'label']]
    
    # Normalisierung
    scaler = StandardScaler()
    train_df[numeric_columns] = scaler.fit_transform(train_df[numeric_columns])
    val_df[numeric_columns] = scaler.transform(val_df[numeric_columns])
    
    # Datasets erstellen
    train_dataset = create_dataset(train_df, numeric_columns)
    val_dataset = create_dataset(val_df, numeric_columns, is_training=False)
    
    # Model erstellen
    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        model = create_model(len(numeric_columns))
        
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
        
        model.compile(
            optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy', 
                    tf.keras.metrics.AUC(name='auc'), 
                    tf.keras.metrics.Precision(name='precision'),
                    tf.keras.metrics.Recall(name='recall')]
        )
    
    # Callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_auc',
            patience=10,
            mode='max',
            restore_best_weights=True,
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_auc',
            factor=0.2,
            patience=5,
            mode='max',
            min_lr=1e-6,
            verbose=1
        ),
        tf.keras.callbacks.ModelCheckpoint(
            'best_model.h5',
            monitor='val_auc',
            mode='max',
            save_best_only=True,
            verbose=1
        )
    ]
    
    # Training
    print("\nStarting training...")
    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=val_dataset,
        callbacks=callbacks,
        verbose=1
    )
    
    # Evaluierung
    val_metrics = model.evaluate(val_dataset)
    print("\nValidation Metrics:")
    for name, value in zip(model.metrics_names, val_metrics):
        print(f"{name}: {value:.4f}")
    
    # Vorhersagen
    y_pred = model.predict(val_dataset)
    y_pred_classes = (y_pred > 0.5).astype("int32")
    y_true = val_df['label'].values
    
    # Metriken
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred_classes))
    
    # ROC AUC
    roc_auc = roc_auc_score(y_true, y_pred)
    print(f"ROC AUC: {roc_auc:.4f}")
    
    # Confusion Matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion_matrix(y_true, y_pred_classes), 
                annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.savefig('confusion_matrix.png')
    plt.close()
    
    # Training History
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.plot(history.history['accuracy'], label='Training')
    plt.plot(history.history['val_accuracy'], label='Validation')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 3, 2)
    plt.plot(history.history['loss'], label='Training')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 3, 3)
    plt.plot(history.history['auc'], label='Training')
    plt.plot(history.history['val_auc'], label='Validation')
    plt.title('Model AUC')
    plt.xlabel('Epoch')
    plt.ylabel('AUC')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()
    
    # Model speichern
    model.save('final_model.h5')
    print("\nTraining completed. Model and visualizations saved.")

if __name__ == "__main__":
    main()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)

Starting training...
Epoch 1/100
126/126 [==============================] - ETA: 0s - loss: 0.7532 - accuracy: 0.6196 - auc: 0.6683 - precision: 0.6149 - recall: 0.6398
Epoch 1: val_auc improved from -inf to 0.85281, saving model to best_model.h5
126/126 [==============================] - 26s 136ms/step - loss: 0.7532 - accuracy: 0.6196 - auc: 0.6683 - precision: 0.6149 - recall: 0.6398 - val_loss: 0.6876 - val_accuracy: 0.5508 - val_auc: 0.8528 - val_precision: 0.5268 - val_recall: 0.9980 - lr: 0.0010
Epoch 2/100
126/126 [==============================] - ETA: 0s - loss: 0.5773 - accuracy: 0.7192 - auc: 0.7947 - precision: 0.7134 - recall: 0.7329
Epoch 2: val_auc did not improve from 0.85281
126/126 [==============================] - 18s 123ms/step - loss: 0.5773 - accuracy: 0.7192 - auc: 0.7947 - precision: 0.7134 - recall: 0.7329 - val_loss: 0.5644 - val_accuracy: 0.6982 - val_auc:

## hybrid_model_optimized_final_new.h5

In [1]:
                                                       ##### Asle jens ine ###########




import tensorflow as tf
from tensorflow.keras import layers, models, Input, mixed_precision
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, accuracy_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

# Load the DataFrame
file_path = r"C:\Users\meh91075\Downloads\AutoSplit_all_in_one_dataset_05112024.csv"
#"C:\Users\meh91075\Downloads\merged_features_filtered.csv"
df = pd.read_csv(file_path)

# Entferne label_standard
df = df.drop('label_standard', axis=1)


print(f"Original CSV data loaded. Number of rows: {len(df)}")
print(f"Original distribution of labels:")
print(df['label_majority'].value_counts())

# Optimize GPU configuration
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        print(e)

# Activate Mixed Precision
if tf.test.is_built_with_cuda():
    policy = mixed_precision.Policy('mixed_float16')
    mixed_precision.set_global_policy(policy)
    print("Mixed precision policy set to mixed_float16")
else:
    print("CUDA not available, using default precision")




# Pfade und Konfiguration
base_dir =  r"C:\Users\meh91075\Desktop\DataSet(S1)"
filtered_dir = os.path.join(base_dir, 'New_filtered_data_label0')
non_filtered_dir = os.path.join(base_dir, 'New_non_filtered_data')



img_height, img_width = 224, 224
batch_size = 32

def get_image_paths(directory):
    return [os.path.join(root, file) for root, _, files in os.walk(directory) 
            for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

filtered_images = get_image_paths(filtered_dir)
non_filtered_images = get_image_paths(non_filtered_dir)
print(f"Found images: Filtered: {len(filtered_images)}, Not filtered: {len(non_filtered_images)}")

def extract_filename(path):
    basename = os.path.basename(path)
    return basename.replace('_ISO', '').replace('_ISOAlt', '').split('.')[0]

image_df = pd.DataFrame({
    'image_path': filtered_images + non_filtered_images,
    'label': [1] * len(filtered_images) + [0] * len(non_filtered_images),
    'filename': [extract_filename(path) for path in filtered_images + non_filtered_images]
})

print("Sample of extracted filenames:")
print(image_df['filename'].head())

# Adjusted merge process
merged_df = pd.merge(df, image_df[['filename', 'image_path']], on='filename', how='left')
merged_df['label'] = merged_df['label_majority'].map({1: 0, 2: 0, 3: 0, 4: 1})
merged_df['has_image'] = merged_df['image_path'].notna()

# Filter only entries with associated images
merged_df_with_images = merged_df[merged_df['has_image']]

# Undersampling of the majority class
majority_class = merged_df_with_images[merged_df_with_images['label'] == 0]
minority_class = merged_df_with_images[merged_df_with_images['label'] == 1]

majority_downsampled = majority_class.sample(n=2000, random_state=42)
balanced_df = pd.concat([majority_downsampled, minority_class])

print(f"After undersampling: Number of rows: {len(balanced_df)}")
print("Distribution of labels after undersampling:")
print(balanced_df['label'].value_counts())

# Train-Test-Split
train_df, val_df = train_test_split(balanced_df, test_size=0.2, stratify=balanced_df['label'], random_state=42)
print(f"Training data: {len(train_df)}, Validation data: {len(val_df)}")

# Determine the numeric columns
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_columns = [col for col in numeric_columns if col not in ['label_majority', 'label']]

# Normalization of numeric data
scaler = StandardScaler()
train_df[numeric_columns] = scaler.fit_transform(train_df[numeric_columns])
val_df[numeric_columns] = scaler.transform(val_df[numeric_columns])

# Optimized data generator function with data augmentation
def create_dataset(dataframe, is_training=True):
    def parse_function(filename, label, *numeric_data):
        img = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [img_height, img_width])
        img = tf.keras.applications.resnet_v2.preprocess_input(img)
        
        if is_training:
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_brightness(img, max_delta=0.2)
            img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
            img = tf.image.random_saturation(img, lower=0.8, upper=1.2)
            img = tf.image.random_hue(img, max_delta=0.1)
        
        numeric_data = tf.convert_to_tensor(numeric_data, dtype=tf.float32)
        numeric_data = tf.squeeze(numeric_data)
        return (img, numeric_data), label

    dataset = tf.data.Dataset.from_tensor_slices((
        dataframe['image_path'].values,
        dataframe['label'].values,
        dataframe[numeric_columns].values
    ))
    
    dataset = dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    
    if is_training:
        dataset = dataset.shuffle(buffer_size=len(dataframe)).repeat()
    else:
        dataset = dataset.repeat()
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

# Create the datasets
train_dataset = create_dataset(train_df)
val_dataset = create_dataset(val_df, is_training=False)

# Calculate steps per epoch
steps_per_epoch = len(train_df) // batch_size
validation_steps = len(val_df) // batch_size

# Optimized model definition
def create_optimized_hybrid_model(img_height, img_width, num_numeric_features):
    # Image processing branch (CNN)
    img_input = Input(shape=(img_height, img_width, 3))
    
    # Use a pre-trained model for feature extraction
    base_model = tf.keras.applications.ResNet50V2(include_top=False, weights='imagenet', input_tensor=img_input)
    
    # Freeze the pre-trained layers
    base_model.trainable = False
    
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    # Numerical data branch (MLP)
    num_input = Input(shape=(num_numeric_features,))
    y = layers.Dense(128, activation='relu')(num_input)
    y = layers.BatchNormalization()(y)
    y = layers.Dropout(0.3)(y)
    y = layers.Dense(256, activation='relu')(y)
    y = layers.BatchNormalization()(y)
    y = layers.Dropout(0.4)(y)
    y = layers.Dense(128, activation='relu')(y)
    y = layers.BatchNormalization()(y)
    y = layers.Dropout(0.3)(y)
    
    # Combining the branches
    combined = layers.concatenate([x, y])
    
    # Final layers
    z = layers.Dense(256, activation='relu')(combined)
    z = layers.BatchNormalization()(z)
    z = layers.Dropout(0.5)(z)
    z = layers.Dense(128, activation='relu')(z)
    z = layers.BatchNormalization()(z)
    z = layers.Dropout(0.5)(z)
    
    # Output layer
    output = layers.Dense(1, activation='sigmoid')(z)
    
    model = models.Model(inputs=[img_input, num_input], outputs=output)
    return model

# Learning Rate Logger Callback
class LearningRateLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            lr = self.model.optimizer.learning_rate
            if isinstance(lr, tf.keras.optimizers.schedules.LearningRateSchedule):
                lr = lr(self.model.optimizer.iterations)
            logs['lr'] = tf.keras.backend.get_value(lr)

# Custom ReduceLROnPlateau Callback
class CustomReduceLROnPlateau(tf.keras.callbacks.Callback):
    def __init__(self, factor=0.1, patience=10, min_lr=0, monitor='val_loss', mode='min'):
        super(CustomReduceLROnPlateau, self).__init__()
        self.factor = factor
        self.patience = patience
        self.min_lr = min_lr
        self.monitor = monitor
        self.mode = mode
        self.best = np.Inf if mode == 'min' else -np.Inf
        self.wait = 0
        self.best_weights = None

    def on_train_begin(self, logs=None):
        self.best = np.Inf if self.mode == 'min' else -np.Inf
        self.wait = 0
        self.best_weights = None

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        current = logs.get(self.monitor)
        if current is None:
            return
        
        if self.mode == 'min' and current < self.best:
            self.best = current
            self.wait = 0
            self.best_weights = self.model.get_weights()
        elif self.mode == 'max' and current > self.best:
            self.best = current
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
        
        if self.wait >= self.patience:
            current_lr = self.model.optimizer.learning_rate
            if isinstance(current_lr, tf.keras.optimizers.schedules.LearningRateSchedule):
                current_lr = current_lr(self.model.optimizer.iterations)
            current_lr = float(current_lr.numpy())
            
            if current_lr > self.min_lr:
                new_lr = max(current_lr * self.factor, self.min_lr)
                new_lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                    initial_learning_rate=new_lr,
                    decay_steps=1000,
                    decay_rate=0.9,
                    staircase=True
                )
                self.model.optimizer.learning_rate = new_lr_schedule
                print(f'\nEpoch {epoch+1}: ReduceLROnPlateau reducing learning rate to {new_lr}.')
                self.wait = 0
                self.model.set_weights(self.best_weights)

# Compile and train
initial_learning_rate = 1e-3
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=1000, decay_rate=0.9, staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Multi-GPU Strategy
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = create_optimized_hybrid_model(img_height, img_width, len(numeric_columns))
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc'), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )

# Callbacks
reduce_lr = CustomReduceLROnPlateau(monitor='val_auc', factor=0.2, patience=5, min_lr=1e-6, mode='max')
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_auc', mode='max')
lr_logger = LearningRateLogger()

# Training
history = model.fit(
    train_dataset,
    epochs=100,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_dataset,
    validation_steps=validation_steps,
    callbacks=[reduce_lr, model_checkpoint, lr_logger]
)

# Save model
model.save('hybrid_model_optimized_final_new.h5')

# Evaluation
val_loss, val_accuracy, val_auc, val_precision, val_recall = model.evaluate(val_dataset, steps=validation_steps)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation AUC: {val_auc:.4f}")
print(f"Validation Precision: {val_precision:.4f}")
print(f"Validation Recall: {val_recall:.4f}")

# Predictions for the validation dataset
y_pred = model.predict(val_dataset, steps=validation_steps)
y_pred_classes = (y_pred > 0.5).astype("int32")
y_true = np.concatenate([y for _, y in val_dataset.take(validation_steps)], axis=0)

# Confusion matrix and classification report
conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_report = classification_report(y_true, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

# Visualization of training history
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(history.history['auc'], label='Training AUC')
plt.plot(history.history['val_auc'], label='Validation AUC')
plt.title('Model AUC')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.legend()

plt.tight_layout()
plt.savefig('training_history.png')
plt.close()

# Visualization of learning rate
plt.figure(figsize=(10, 5))
plt.plot(history.history['lr'])
plt.title('Learning Rate over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.yscale('log')
plt.savefig('learning_rate.png')
plt.close()

# Visualization of confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig('confusion_matrix.png')
plt.close()

print("Visualizations have been saved as PNG files.")

# Save predictions and true labels
predictions_df = pd.DataFrame({
    'True_Label': y_true,
    'Predicted_Probability': y_pred.flatten(),
    'Predicted_Class': y_pred_classes.flatten()
})
predictions_df.to_csv('predictions.csv', index=False)
print("Predictions have been saved in 'predictions.csv'.")

# Model summary
model.summary()

# Save model architecture as image
tf.keras.utils.plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)
print("Model architecture has been saved as 'model_architecture.png'.")

# Calculate and output additional metrics
# ROC AUC
roc_auc = roc_auc_score(y_true, y_pred)
print(f"ROC AUC: {roc_auc:.4f}")

# Precision-Recall AUC
pr_auc = average_precision_score(y_true, y_pred)
print(f"Precision-Recall AUC: {pr_auc:.4f}")

# F1-Score at various thresholds
thresholds = np.arange(0, 1.01, 0.01)
f1_scores = [f1_score(y_true, y_pred > threshold) for threshold in thresholds]
best_threshold = thresholds[np.argmax(f1_scores)]
best_f1 = max(f1_scores)
print(f"Best F1-Score: {best_f1:.4f} at threshold {best_threshold:.2f}")

# Visualization of Precision-Recall curve
precision, recall, _ = precision_recall_curve(y_true, y_pred)
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.')
plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.savefig('precision_recall_curve.png')
plt.close()
print("Precision-Recall curve has been saved as 'precision_recall_curve.png'.")

# Visualization of ROC curve
fpr, tpr, _ = roc_curve(y_true, y_pred)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.savefig('roc_curve.png')
plt.close()
print("ROC curve has been saved as 'roc_curve.png'.")

# Save training history
with open('training_history.txt', 'w') as f:
    for key, values in history.history.items():
        f.write(f"{key}: {values}\n")
print("Training history has been saved in 'training_history.txt'.")

# Analyze misclassified examples
misclassified = np.where(y_pred_classes.flatten() != y_true)[0]
print(f"\nNumber of misclassified examples: {len(misclassified)}")

if len(misclassified) > 0:
    print("\nDetails of some misclassified examples:")
    for i in misclassified[:5]:  # Show details for the first 5 misclassified examples
        true_label = y_true[i]
        pred_label = y_pred_classes[i][0]
        pred_prob = y_pred[i][0]
        print(f"Example {i}: True Label: {true_label}, Predicted Label: {pred_label}, Predicted Probability: {pred_prob:.4f}")

# Model performance across different thresholds
accuracies = [accuracy_score(y_true, y_pred > threshold) for threshold in thresholds]

plt.figure(figsize=(10, 6))
plt.plot(thresholds, accuracies, label='Accuracy')
plt.plot(thresholds, f1_scores, label='F1-Score')
plt.axvline(x=0.5, color='r', linestyle='--', label='Default Threshold (0.5)')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Model Performance vs. Classification Threshold')
plt.legend()
plt.savefig('performance_vs_threshold.png')
plt.close()
print("Performance vs. Threshold plot has been saved as 'performance_vs_threshold.png'.")

# Optional: SHAP Analysis
try:
    import shap
    
    # Select a subset of validation data for SHAP analysis
    num_explain = 100
    explain_dataset = val_dataset.take(num_explain).unbatch()
    background_dataset = train_dataset.take(num_explain).unbatch()
    
    # Create an Explainer
    explainer = shap.DeepExplainer(model, background_dataset)
    
    # Calculate SHAP values
    shap_values = explainer.shap_values(explain_dataset)
    
    # Visualize SHAP values
    shap.summary_plot(shap_values[0], plot_type="bar", feature_names=numeric_columns, show=False)
    plt.savefig('shap_summary.png')
    plt.close()
    print("SHAP summary has been saved as 'shap_summary.png'.")
except ImportError:
    print("SHAP is not installed. Skipping SHAP analysis.")

print("\nAll analyses and visualizations are complete.")

# Summary of key metrics
print("\nSummary of key metrics:")
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation AUC: {val_auc:.4f}")
print(f"Validation Precision: {val_precision:.4f}")
print(f"Validation Recall: {val_recall:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"Precision-Recall AUC: {pr_auc:.4f}")
print(f"Best F1-Score: {best_f1:.4f} at threshold {best_threshold:.2f}")

# Save summary to a text file
with open('model_summary.txt', 'w') as f:
    f.write("Summary of key metrics:\n")
    f.write(f"Validation Accuracy: {val_accuracy:.4f}\n")
    f.write(f"Validation AUC: {val_auc:.4f}\n")
    f.write(f"Validation Precision: {val_precision:.4f}\n")
    f.write(f"Validation Recall: {val_recall:.4f}\n")
    f.write(f"ROC AUC: {roc_auc:.4f}\n")
    f.write(f"Precision-Recall AUC: {pr_auc:.4f}\n")
    f.write(f"Best F1-Score: {best_f1:.4f} at threshold {best_threshold:.2f}\n")

print("\nA summary of key metrics has been saved in 'model_summary.txt'.")
print("\nThe script has been executed successfully. All results and visualizations have been saved.")



Original CSV data loaded. Number of rows: 19274
Original distribution of labels:
label_majority
2    10850
1     3485
4     2745
3     2194
Name: count, dtype: int64
1 Physical GPUs, 1 Logical GPUs
Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  Quadro P5000, compute capability 6.1
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once
Mixed precision policy set to mixed_float16
Found images: Filtered: 5664, Not filtered: 27994
Sample of extracted filenames:
0       00042065
1    00042065Alt
2       00043749
3    00043749Alt
4       00080967
Name: filename, dtype: object
After undersampling: Number of rows: 4499
Distribution of labels after undersampling:
label
1    2499
0    2000
Name: count, dtype: int6

AttributeError: module 'pydot' has no attribute 'InvocationException'